In [4]:
import numpy as np
import pandas as pd
from collections import defaultdict
import pymongo
import dill
import os

In [5]:
FOLDER = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Predictions/"

In [50]:
def filter_by_str(s, files, exclude=False):   
    return  [f for f in files if (s in f) != exclude]

In [81]:
files = os.listdir(FOLDER)
print(len(files))
files = filter_by_str("_VD_", files)
files = filter_by_str("_VD_", files)
print(len(files))
files = filter_by_str("2019", files, exclude=True)
print(len(files))

78
38
16


In [82]:
sorted(files)

['TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG_PREDS_.dill',
 'TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG_YS_.dill',
 'TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG_FIXED_PREDS_.dill',
 'TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG_FIXED_YS_.dill',
 'TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS_FIXED_PREDS_.dill',
 'TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS_FIXED_YS_.dill',
 'TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_PREDS_.dill',
 'TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_YS_.dill',
 'TEST_SC_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG_PREDS_.dill',
 'TEST_SC_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG_YS_.dill',
 'TEST_SC_TAGGING_VD_CRF_MOST_COMMON_TAG_PREDS_.dill',
 'TEST_SC_TAGGING_VD_CRF_MOST_COMMON_TAG_YS_.dill',
 'TEST_SC_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS_PREDS_.dill',
 'TEST_SC_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS_YS_.dill',
 'TEST_SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_PREDS_.dill',
 'TEST_SC

In [83]:
sc_files = filter_by_str("_SC_", files)
cb_files = filter_by_str("_CB_", files)
len(cb_files), len(sc_files)

(8, 8)

In [84]:
filters = {
    "PERCEPTRON":"PERCEPTRON",
    "WINDOW_CLASSIFIER":"WINDOW_CLASSIFIER",
    "CRF": "_CRF_",
    "HMM" : "_HMM_"
}

In [95]:
from pprint import pprint

def load_predictions(input_files):
    input_files = cb_files
    algo2preds = dict()
    pred_files = filter_by_str("_PREDS", input_files)
    for algo_name, fltr in filters.items():
        f_files = filter_by_str(fltr, pred_files)
        assert len(f_files) == 1
        fname = FOLDER + f_files[0]
        with open(fname) as f:
            algo2preds[algo_name] = dill.load(f)
    return algo2preds

In [96]:
cb_preds = load_predictions(cb_files)
sc_preds = load_predictions(sc_files)

In [97]:
with open(FOLDER + "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_YS_.dill") as f:
    cb_ysbytag = dill.load(f)
    
with open(FOLDER + "TEST_SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_YS_.dill") as f:
    sc_ysbytag = dill.load(f)

In [99]:
# sorted(files)

In [61]:
from scipy import stats

In [62]:
def get_all_preds(predsbytag):
    all_p = []
    for k, vals in sorted(predsbytag.items(), key = lambda tpl: tpl[0]):
        all_p.extend(vals)
    return all_p

In [76]:
def compute_p_value_binomial_test(ysbytag, predsbytaga, predsbytagb):
    assert len(ysbytag.keys()) == len(predsbytaga.keys())
    assert len(ysbytag.keys()) == len(predsbytagb.keys())

    first = list(ysbytag.keys())[0]
    assert len(ysbytag[first]) == len(predsbytaga[first])
    assert len(ysbytag[first]) == len(predsbytagb[first])

    ys = get_all_preds(ysbytag)
    aas = get_all_preds(predsbytaga)
    bbs = get_all_preds(predsbytagb)

    assert len(ys) == len(aas) == len(bbs)

    successes = defaultdict(int)
    for y,a,b in zip(ys,aas,bbs):    
        if a != b:
            if a == y:
                successes["a"] += 1
            else:
                successes["b"] += 1

    p_value = stats.binom_test(successes["a"], sum(successes.values()), p=0.5, alternative='two-sided')
    return p_value

In [77]:
compute_p_value_binomial_test(ysbytag, predsbytaga, predsbytagb)

0.41714359087871278

In [102]:
ysbytag = cb_ysbytag
algo2preds = cb_preds

matrix = dict()
for algo_name_a, predsbytaga in algo2preds.items():
    for algo_name_b, predsbytagb in algo2preds.items():
        if algo_name_a == algo_name_b:
            continue
        pval = compute_p_value_binomial_test(ysbytag, predsbytaga, predsbytagb)
        print algo_name_a.ljust(20), algo_name_b.ljust(20), pval
        matrix[(algo_name_a, algo_name_b)] = pval

WINDOW_CLASSIFIER    PERCEPTRON           0.599071512167
WINDOW_CLASSIFIER    CRF                  0.165544403238
WINDOW_CLASSIFIER    HMM                  6.53926360882e-216
PERCEPTRON           WINDOW_CLASSIFIER    0.599071512167
PERCEPTRON           CRF                  0.395331705872
PERCEPTRON           HMM                  1.84027241494e-204
CRF                  WINDOW_CLASSIFIER    0.165544403238
CRF                  PERCEPTRON           0.395331705872
CRF                  HMM                  3.40817664793e-186
HMM                  WINDOW_CLASSIFIER    6.53926360882e-216
HMM                  PERCEPTRON           1.84027241494e-204
HMM                  CRF                  3.40817664793e-186


In [104]:
sorted(matrix.items(), key = lambda tpl: -tpl[1])

[(('PERCEPTRON', 'WINDOW_CLASSIFIER'), 0.59907151216703736),
 (('WINDOW_CLASSIFIER', 'PERCEPTRON'), 0.59907151216703736),
 (('PERCEPTRON', 'CRF'), 0.39533170587228922),
 (('CRF', 'PERCEPTRON'), 0.39533170587228922),
 (('CRF', 'WINDOW_CLASSIFIER'), 0.16554440323794242),
 (('WINDOW_CLASSIFIER', 'CRF'), 0.16554440323794242),
 (('HMM', 'CRF'), 3.4081766479273455e-186),
 (('CRF', 'HMM'), 3.4081766479273455e-186),
 (('PERCEPTRON', 'HMM'), 1.8402724149422639e-204),
 (('HMM', 'PERCEPTRON'), 1.8402724149422639e-204),
 (('WINDOW_CLASSIFIER', 'HMM'), 6.5392636088222868e-216),
 (('HMM', 'WINDOW_CLASSIFIER'), 6.5392636088222868e-216)]